In [ ]:
import pandas as pd
from genepy.utils import helper as h
import numpy as np

In [ ]:
old_rsem_genes_tpm = pd.read_csv("gs://fc-secure-d6345f68-81ef-475e-95e0-1e3a8e925f61/ddaac348-0b3e-4f3a-b85e-603f1f090d43/RNA_aggregate/23ef35d1-c4e6-47e5-98b1-b966a859f7f9/call-rsem_aggregate_results/all.rsem_genes_expected_count.txt.gz", sep='\t')
old_rsem_genes_tpm = old_rsem_genes_tpm[(old_rsem_genes_tpm.sum(1) != 0) & (old_rsem_genes_tpm.var(1) != 0)]
old_rsem_genes_tpm

In [ ]:
final_rsem_genes_tpm = pd.read_csv("gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd50046a968/f698468d-ac97-4303-9ef2-ca25058a7382/RNA_aggregate/77ce446d-f9e6-43d8-b411-3a31241988d7/call-rsem_aggregate_results/all.rsem_genes_tpm.txt.gz", sep='\t')
final_rsem_genes_tpm = final_rsem_genes_tpm[(final_rsem_genes_tpm.sum(1) != 0) & (final_rsem_genes_tpm.var(1) != 0)]
final_rsem_genes_tpm

In [ ]:
r = [i.split(".")[0] for i in final_rsem_genes_tpm['gene_id']]
dup = h.dups(r)

In [ ]:
mybiomart = h.generateGeneNames(ensemble_server='http://nov2020.archive.ensembl.org/biomart', useCache=True)
# creating renaming index, keeping top name first
gene_rename = {}
for _, i in mybiomart.iterrows():
    if i.ensembl_gene_id not in gene_rename:
        gene_rename.update(
            {i.ensembl_gene_id: i.hgnc_symbol + " (" + i.ensembl_gene_id + ")"}
        )
protcod_rename = {}
for _, i in mybiomart[
    (~mybiomart.entrezgene_id.isna()) & (mybiomart.gene_biotype == "protein_coding")
].iterrows():
    if i.ensembl_gene_id not in protcod_rename:
        protcod_rename.update(
            {
                i.ensembl_gene_id: i.hgnc_symbol
                + " ("
                + str(int(i.entrezgene_id))
                + ")"
            }
        )

In [ ]:
len(mybiomart[mybiomart.ensembl_gene_id.isin(dup)])

In [ ]:
startswith(mybiomart[mybiomart.ensembl_gene_id == 'ENSG00000002586'].hgnc_symbol)

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()

genes_tpm_logp1 = tc.get(name='expression-d035', version=21, file='genes_tpm_logp1')

In [ ]:
from gsheets import Sheets
SHEETCREDS = "../.credentials.json"
MY_ID = "~/.client_secret.json"
MYSTORAGE_ID = "~/.storage.json"
REFSHEET_URL = "https://docs.google.com/spreadsheets/d/1Pgb5fIClGnErEqzxpU7qqX6ULpGTDjvzWwDN8XUJKIY"

t = Sheets.from_files(MY_ID, MYSTORAGE_ID).get(REFSHEET_URL).sheets[0].to_frame(index_col=0)

for d in dup:
    s = final_rsem_genes_tpm[final_rsem_genes_tpm.gene_id.str.startswith(d)].iloc[:, 2:]
    if s.duplicated().sum() != 1:
        print('DUP ENSEMBL ID: ', d)
        diff = s.diff().T
        idx = diff.columns[-1]
        for i in diff[diff[idx] != 0].index:
            print('non-NA CDS id: ', i)
            arxs = t.loc[i].arxspan_id
            colname = mybiomart[mybiomart.ensembl_gene_id == d].hgnc_symbol + " (" + d + ")"
            print('expression level in previous genes_tpm_logp1: ', float(genes_tpm_logp1.loc[arxs, colname]))
            print('expression level in current aggregation: ')
            print(final_rsem_genes_tpm[final_rsem_genes_tpm.gene_id.str.startswith(d)].gene_id.tolist()[0], np.log2(s[i].tolist()[0] + 1))
            print(final_rsem_genes_tpm[final_rsem_genes_tpm.gene_id.str.startswith(d)].gene_id.tolist()[1], np.log2(s[i].tolist()[1] + 1))
            print("combined: ", np.log2(s[i].tolist()[0] + s[i].tolist()[1] + 1), '\n')
            

In [ ]:
from gsheets import Sheets
SHEETCREDS = "../.credentials.json"
MY_ID = "~/.client_secret.json"
MYSTORAGE_ID = "~/.storage.json"
REFSHEET_URL = "https://docs.google.com/spreadsheets/d/1Pgb5fIClGnErEqzxpU7qqX6ULpGTDjvzWwDN8XUJKIY"

genes_tpm = tc.get(name='expression-d035', version=21, file='genes_tpm')
t = Sheets.from_files(MY_ID, MYSTORAGE_ID).get(REFSHEET_URL).sheets[0].to_frame(index_col=0)

for d in dup:
    s = final_rsem_genes_tpm[final_rsem_genes_tpm.gene_id.str.startswith(d)].iloc[:, 2:]
    if s.duplicated().sum() != 1:
        print('DUP ENSEMBL ID: ', d)
        diff = s.diff().T
        idx = diff.columns[-1]
        for i in diff[diff[idx] != 0].index:
            print('non-NA CDS id: ', i)
            arxs = t.loc[i].arxspan_id
            colname = mybiomart[mybiomart.ensembl_gene_id == d].hgnc_symbol + " (" + d + ")"
            print('expression level in previous genes_tpm: ', float(genes_tpm.loc[arxs, colname]))
            print('expression level in current aggregation: ')
            print(final_rsem_genes_tpm[final_rsem_genes_tpm.gene_id.str.startswith(d)].gene_id.tolist()[0], s[i].tolist()[0])
            print(final_rsem_genes_tpm[final_rsem_genes_tpm.gene_id.str.startswith(d)].gene_id.tolist()[1], s[i].tolist()[1])
            print("combined: ", s[i].tolist()[0] + s[i].tolist()[1], '\n')
            

In [ ]:
final_rsem_genes_tpm[final_rsem_genes_tpm.gene_id.str.startswith('ENSG00000002586')].iloc[:, 2:]

In [ ]:
final_rsem_genes_tpm[final_rsem_genes_tpm.gene_id.str.startswith('ENSG00000002586')].loc[:, 'CDS-Et3K1I']

In [ ]:
final_rsem_genes_tpm.loc[23]

In [ ]:
mybiomart[mybiomart.ensembl_gene_id == 'ENSG00000002586']

In [ ]:
genes_tpm.loc['ACH-001113', 'CD99 (ENSG00000002586)']

In [ ]:
rsem_genes_expected_count = pd.read_csv("gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd50046a968/f698468d-ac97-4303-9ef2-ca25058a7382/RNA_aggregate/77ce446d-f9e6-43d8-b411-3a31241988d7/call-rsem_aggregate_results/all.rsem_genes_expected_count.txt.gz", sep='\t')
rsem_genes_expected_count = rsem_genes_expected_count[(rsem_genes_expected_count.sum(1) != 0) & (rsem_genes_expected_count.var(1) != 0)]
rsem_genes_expected_count


In [ ]:
rsem_genes_expected_count[rsem_genes_expected_count.gene_id.str.endswith('_PAR_Y')]

In [ ]:
r = [i.split(".")[0] for i in rsem_genes_expected_count['gene_id']]
dup = h.dups(r)

In [ ]:
dup

In [ ]:
from gsheets import Sheets
SHEETCREDS = "../.credentials.json"
MY_ID = "~/.client_secret.json"
MYSTORAGE_ID = "~/.storage.json"
REFSHEET_URL = "https://docs.google.com/spreadsheets/d/1Pgb5fIClGnErEqzxpU7qqX6ULpGTDjvzWwDN8XUJKIY"

genes_expected_count = tc.get(name='expression-d035', version=21, file='genes_expected_count')
t = Sheets.from_files(MY_ID, MYSTORAGE_ID).get(REFSHEET_URL).sheets[0].to_frame(index_col=0)

for d in dup:
    s = rsem_genes_expected_count[rsem_genes_expected_count.gene_id.str.startswith(d)].iloc[:, 2:]
    if s.duplicated().sum() != 1:
        print('DUP ENSEMBL ID: ', d)
        diff = s.diff().T
        idx = diff.columns[-1]
        for i in diff[diff[idx] != 0].index:
            print('non-NA CDS id: ', i)
            arxs = t.loc[i].arxspan_id
            colname = mybiomart[mybiomart.ensembl_gene_id == d].hgnc_symbol + " (" + d + ")"
            print('expression level in previous genes_tpm: ', float(genes_expected_count.loc[arxs, colname]))
            print('expression level in current aggregation: ')
            print(rsem_genes_expected_count[rsem_genes_expected_count.gene_id.str.startswith(d)].gene_id.tolist()[0], s[i].tolist()[0])
            print(rsem_genes_expected_count[rsem_genes_expected_count.gene_id.str.startswith(d)].gene_id.tolist()[1], s[i].tolist()[1])
            print("combined: ", s[i].tolist()[0] + s[i].tolist()[1], '\n')
            

In [ ]:
# rsem_genes_expected_count = rsem_genes_expected_count.set_index("gene_id")
par_y_drop = []
for d in dup:
    idx = rsem_genes_expected_count[(rsem_genes_expected_count.index.str.startswith(d)) & (~rsem_genes_expected_count.index.str.endswith('_PAR_Y'))].index[0]
    idx_par_y = rsem_genes_expected_count[(rsem_genes_expected_count.index.str.startswith(d)) & (rsem_genes_expected_count.index.str.endswith('_PAR_Y'))].index[0]
    rsem_genes_expected_count.loc[idx] = rsem_genes_expected_count.loc[[idx, idx_par_y], :].sum()
    par_y_drop.append(idx_par_y)
rsem_genes_expected_count = rsem_genes_expected_count.drop(index=par_y_drop)

In [ ]:
idx

In [ ]:
rsem_genes_expected_count.loc['ENSG00000002586.20', 'transcript_id(s)']

In [ ]:
rsem_transcripts_expected_count = pd.read_csv("gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd50046a968/f698468d-ac97-4303-9ef2-ca25058a7382/RNA_aggregate/77ce446d-f9e6-43d8-b411-3a31241988d7/call-rsem_aggregate_results/all.rsem_transcripts_expected_count.txt.gz", sep='\t')
rsem_transcripts_expected_count = rsem_transcripts_expected_count[(rsem_transcripts_expected_count.sum(1) != 0) & (rsem_transcripts_expected_count.var(1) != 0)]
rsem_transcripts_expected_count


In [ ]:
rsem_transcripts_expected_count[rsem_transcripts_expected_count.gene_id.str.startswith('ENSG00000002586.20')]

In [ ]:
r = [i.split(".")[0] for i in rsem_transcripts_expected_count.transcript_id]
dup = h.dups(r)
dup

In [ ]:
rsem_transcripts_expected_count[rsem_transcripts_expected_count.gene_id.str.startswith('ENSG00000010030')]

In [ ]:
old_rsem_transcripts_expected_count = pd.read_csv("gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd50046a968/efcf68f6-73a8-4d28-9200-99e2f8e9a84d/rsem_aggregate_results_workflow/a75ed264-7e9d-4c4d-b74b-7a74adc71e69/call-rsem_aggregate_results/all.rsem_transcripts_expected_count.txt.gz", sep='\t')
old_rsem_transcripts_expected_count = old_rsem_transcripts_expected_count[(old_rsem_transcripts_expected_count.sum(1) != 0) & (old_rsem_transcripts_expected_count.var(1) != 0)]
old_rsem_transcripts_expected_count



In [ ]:
transcripts_expected_count = tc.get(name='expression-d035', version=21, file='transcripts_expected_count')


In [ ]:
transcripts_expected_count.loc['ACH-001113', 'CD99 (ENST00000381177)']

In [ ]:
transcripts_expected_count.columns.endswith('ENSG00000002586')

In [ ]:
transcripts_expected_count

In [ ]:
final_rsem_genes_tpm[final_rsem_genes_tpm.gene_id.str.startswith("ENSG00000277203")]

In [ ]:
old_rsem_genes_tpm[old_rsem_genes_tpm.gene_id.str.startswith("ENSG00000277203")]

In [ ]:
mybiomart[mybiomart.hgnc_symbol == 'F8A1']

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()

CCLE_expression_old = tc.get(name='internal-22q1-1778', version=24, file='CCLE_expression')

In [ ]:
CCLE_expression_new = tc.get(name='tentative-virtual-d84e', version=53, file='CCLE_expression')

In [ ]:
CCLE_expression_new

In [ ]:
from depmapomics.qc.test_compare_to_ref_release import (
    FILE_ATTRIBUTES_PAIRED,
    PREV_RELEASE,
    NEW_RELEASE,
    data,
    get_both_releases_from_taiga,
)

def get_data_stack(file, number_of_points=100000, random_state=0):
    data1, data2 = CCLE_expression_old, CCLE_expression_new

    row = set(data1.index) & set(data2.index)
    col = set(data1.columns) & set(data2.columns)

    data1_stack = data1.loc[row, col].stack()
    data2_stack = data2.loc[row, col].stack()

    if number_of_points > 0:
        data1_stack = data1_stack.sample(number_of_points)

    idx = list(set(data1_stack.index) & set(data2_stack.index))

    data1_stack = data1_stack.loc[idx]
    data2_stack = data2_stack.loc[idx]

    data_stack = pd.concat([data1_stack, data2_stack], axis=1)

    cols = [
        "{:s}.{:d}".format(PREV_RELEASE["name"], PREV_RELEASE["version"]),
        "{:s}.{:d}".format(NEW_RELEASE["name"], NEW_RELEASE["version"]),
    ]
    data_stack.columns = cols
    data_stack.reset_index(inplace=True)
    data_stack.rename(columns={"level_0": "DepMap_ID", "level_1": "gene"}, inplace=True)
    return data_stack, cols

In [ ]:
data_stack, cols = get_data_stack("")
data_stack

In [ ]:
data_stack[(data_stack['internal-22q1-1778.24'] < 2.5) & (data_stack['tentative-virtual-d84e.53'] > 7.5)]

In [ ]:
data_stack, cols = get_data_stack("")
data_stack[(data_stack['internal-22q1-1778.24'] < 2.5) & (data_stack['tentative-virtual-d84e.53'] > 7.5)]

In [ ]:
data_stack, cols = get_data_stack("")
data_stack[(data_stack['internal-22q1-1778.24'] < 2.5) & (data_stack['tentative-virtual-d84e.53'] > 7.5)]

In [ ]:
data_stack, cols = get_data_stack("")
data_stack[(data_stack['internal-22q1-1778.24'] < 2.5) & (data_stack['tentative-virtual-d84e.53'] > 7.5)]

In [ ]:
data_stack, cols = get_data_stack("")
data_stack[(data_stack['internal-22q1-1778.24'] < 2.5) & (data_stack['tentative-virtual-d84e.53'] > 7.5)]

In [ ]:
CCLE_expression = tc.get(name='tentative-virtual-d84e', version=53, file='CCLE_expression')
'RPL36A-HNRNPH2 (100529097)' in CCLE_expression.columns